In [ ]:
import boto3
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook
from sagemaker import get_execution_role
role = get_execution_role()
s3 = boto3.resource('s3')

In [ ]:
# We will be using just hospitals 3 and 4
icusics_db_patients = pd.read_parquet(
    "s3://icusics-db/patients/patients.parquet")
patients = icusics_db_patients[icusics_db_patients['hospital_coded'].isin(
    [3, 4])].copy()

## We need to flag patients with antibiotic treatment to detect sepsis

In [ ]:
antibiotics = [
    'AMIKACINA', 'AMOXICIL.LINA+CLAVULANIC', 'AMOXICIL?LINA+CLAVULANIC',
    'AMOXICIL·LINA+CLAVULANIC', 'AMPICIL.LINA', 'AMPICIL·LINA', 'AZITROMICINA',
    'AZTREONAM', 'CEFAZOLINA', 'CEFEPIMA', 'CEFOTAXIMA', 'CEFTAZIDIMA',
    'CEFTAZIDIMA+AVIBACTAM', 'CEFTOLOZANO+TAZOBACTAM', 'CEFTRIAXONA',
    'CEFUROXIMA', 'CIPROFLOXACINA', 'CLARITROMICINA', 'CLINDAMICINA',
    'CLOXACIL.LINA', 'CLOXACIL·LINA', 'COLIST+TOBRAMI(POLIANTIBIOTICA)',
    'COLISTINA', 'COTRIMOXAZOL', 'COTRIMOXAZOLE',
    'COTRIMOXAZOLE(TRIMETO+SULFAMETOX)', 'DAPTOMICINA', 'DOXICICLINA',
    'ERITROMICINA', 'ERTAPENEM', 'FIDAXOMICINA', 'FOSFOMICINA', 'GENTAMICINA',
    'IMIPENEM+CILASTATINA', 'LEVOFLOXACINO', 'LINEZOLID', 'MEROPENEM',
    'METRONIDAZOL', 'NORFLOXACINO', 'PENICIL·LINA',
    'PIPERACIL.LINA+TAZOBACTAM', 'PIPERACIL·LINA+TAZOBACTAM', 'SULBACTAM',
    'SULFADIAZINA', 'TEICOPLANINA', 'TIGECICLINA', 'TOBRAMICINA', 'VANCOMICINA'
]

d_pharma = pd.read_parquet(f's3://icusics-db/d_pharma/d_pharma.parquet')
d_pharma = d_pharma[d_pharma['hospital_coded'].isin([3, 4])].copy()
d_pharma.loc[:, 'short_name'] = d_pharma['pharmaname'].apply(
    lambda x: x.split(' ')[0])
d_pharma = d_pharma[d_pharma['short_name'].isin(antibiotics)].copy()
antibiotics_id = set(d_pharma['a_pharmaid'])
del d_pharma

In [ ]:
pharma_h3 = pd.read_parquet(f's3://icusics-db/pharma_records/pharma_records_h3.parquet')
pharma_h3 = pharma_h3[pharma_h3['a_pharmaid'].isin(antibiotics_id) & (pharma_h3['time']<=1440)].copy()

pharma_h4 = pd.read_parquet(f's3://icusics-db/pharma_records/pharma_records_h4.parquet')
pharma_h4 = pharma_h4[pharma_h4['a_pharmaid'].isin(antibiotics_id) & (pharma_h4['time']<=1440)].copy()

antibiotics_patients = pd.concat([pharma_h3[['a_patientid']], pharma_h4[['a_patientid']]]).drop_duplicates()

## We also need the sofa for finding septic patients

### Let's start with respiratory sofa

In [ ]:
# First, look for the specific variables
key_chars = 'SaO2/FiO2|SpO2/FiO2'

pafi_result_dummy = d_variables[((d_variables['hospital_coded']==3) | (d_variables['hospital_coded']==4)) & (
    (d_variables['name'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['description'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['abbreviation'].str.contains(key_chars, case=False, na=False)))]
pafi_result_dummy

In [ ]:
# We will be using SaO2/FiO2
def resp(x):
    if x['ventilated'] and x['sao2_fio2'] < 67:
        return 4
    elif x['ventilated'] and x['sao2_fio2'] <= 141:
        return 3
    elif not x['ventilated'] and x['sao2_fio2'] <= 220:
        return 2
    elif not x['ventilated'] and x['sao2_fio2'] < 301:
        return 1
    return 0


# We have identified variable IDs 3030003500, 4030003500 to be the SaO2/FiO2 value
sao2_fio2 = []
for my_bucket_object in tqdm_notebook(bucket.objects.all(),
                                      total=len(list(bucket.objects.all()))):

    if all(x in my_bucket_object.key for x in ['derived_numeric_']):

        boundaries = re.findall('\d+', my_bucket_object.key)
        ub = float(boundaries.pop(1))
        lb = float(boundaries.pop())
        if lb >= min(patients['a_patientid']) or ub <= max(
                patients['a_patientid']):
            # for patid in set(patients['a_patientid'].astype(str)):
            #     if patid>lb and patid<ub:
            chunk = pd.read_parquet(f's3://icusics-db/{my_bucket_object.key}')
            aux = chunk[(chunk['a_variableid'].isin([3030003500, 4030003500]))
                        & (chunk['time'] <= 1440) & (chunk['a_patientid'].isin(
                            set(patients['a_patientid'])))]
            # assert len(aux) > 0
            sao2_fio2.append(aux)
            if len(aux) > 0:
                print('found!')

sao2_fio2 = pd.concat(sao2_fio2)

# After some analysis, we found that variables 3015002262 and 4015003358 correspond to mechanical ventilation
# For variable 3015002262, we have to restrict to the value '12'
# For variable 4015003358, we have to restrict to the values '5', '9' and '11'

observed_categoric = pd.read_parquet(
    's3://icusics-db/observed_categoric/observed_categoric_h3.parquet')
mv_patients_h3 = list(
    set(observed_categoric[(observed_categoric['a_variableid'] == 3015002262)
                           & (observed_categoric['time'] <= 1440) &
                           (observed_categoric['choicecode']
                            == 12)]['a_patientid']))

observed_categoric = pd.read_parquet(
    's3://icusics-db/observed_categoric/observed_categoric_h4.parquet')
mv_patients_h4 = list(
    set(observed_categoric[(observed_categoric['a_variableid'] == 4015003358)
                           & (observed_categoric['time'] <= 1440) &
                           (observed_categoric['choicecode'].isin(
                               [5, 9, 11]))]['a_patientid']))

mv_patients = mv_patients_h3 + mv_patients_h4

In [ ]:
resp_sofa = sao2_fio2.groupby('a_patientid',
                      as_index=False).min()[['a_patientid', 'value']]
resp_sofa.rename(columns={'value': 'sao2_fio2'}, inplace=True)

resp_sofa.loc[:, 'ventilated'] = False
resp_sofa.loc[resp_sofa['a_patientid'].isin(set(mv_patients['a_patientid'])),
                'ventilated'] = True
resp_sofa.loc[:, 'resp_sofa'] = resp_sofa.apply(resp, axis=1)

### Now we go for the renal SOFA

In [ ]:
# We found that observed numeric variables 4010014610 and 3010014610 contain the creatinine
moncat_1 = pd.read_parquet(
    's3://icusics-db/observed_numeric/observed_numeric_h3.parquet')
moncat_2 = pd.read_parquet(
    's3://icusics-db/observed_numeric/observed_numeric_h4.parquet')

moncat = pd.concat([moncat_1, moncat_2])

renal_sofa = moncat[((moncat['time'] > 0) & (moncat['time'] <= 1440))
                     & ((moncat['a_variableid'] == 4010014610)
                        | (moncat['a_variableid'] == 3010014610))].copy()
renal_sofa = renal_sofa.groupby(['a_patientid'], as_index=False).max()

In [ ]:
def func_creat(x):
    if x['value'] >= 5.0:
        return 4
    elif x['value'] >= 3.5 and x['value'] < 5.0:
        return 3
    elif x['value'] >= 2.0 and x['value'] < 3.5:
        return 2
    elif x['value'] >= 1.2 and x['value'] < 2.0:
        return 1
    else:
        return 0
renal_sofa.loc[:, 'renal_sofa'] = renal_sofa.apply(func_creat, axis=1)

### Then, the liver one

In [ ]:
# We found that observed numeric variables 4020004300, 3024000642 and 3010014610 contain the bilirubin

labres_1 = pd.read_parquet('s3://icusics-db/labresults_numeric/labresults_numeric_h3.parquet')
labres_2 = pd.read_parquet('s3://icusics-db/labresults_numeric/labresults_numeric_h4.parquet')

labres = pd.concat([labres_1,labres_2])

liver_sofa = labres[((labres['time'] > 0) & (labres['time'] <= 1440))
                 & ((labres['a_variableid'] == 4020004300)
                    | (labres['a_variableid'] == 3024000642)
                    | (labres['a_variableid'] == 3020004300))]
liver_sofa = liver_sofa.groupby(['a_patientid']).max()

In [ ]:
def func_bill(x):
    if x['value'] >= 12.0:
        return 4
    elif x['value'] >= 6.0:
        return 3
    elif x['value'] >= 2.0:
        return 2
    elif x['value'] >= 1.2:
        return 1
    elif np.isnan(x['value']):
        return np.nan
    else:
        return 0
liver_sofa.loc[:, 'liver_sofa'] = liver_sofa.apply(func_bill, axis=1)

### Let's find the coagulation SOFA

In [ ]:
# We found that observed numeric variables 3024006332 and 4020000110 contain the platelets

coag_sofa = labres[((labres['time'] > 0) & (labres['time'] <= 1440))
                  & ((labres['a_variableid'] == 3024006332)
                     | (labres['a_variableid'] == 4020000110))]
coag_sofa = coag_sofa.groupby(['a_patientid']).max()

In [ ]:
def func_plaq(x):
    if x['value'] < 20.0:
        return 4
    elif x['value'] < 50.0:
        return 3
    elif x['value'] < 100.0:
        return 2
    elif x['value'] < 150.0:
        return 1
    elif np.isnan(x['value']):
        return np.nan
    else:
        return 0
coag_sofa.loc[:, 'coag_sofa'] = coag_sofa.apply(func_plaq, axis=1)

### Then, the CNS SOFA

In [ ]:
# We found that observed numeric variables 4010014625 and 3010014625 contain the platelets

cns_sofa = moncat[((moncat['time'] > 0) & (moncat['time'] <= 1440))
                  & ((moncat['a_variableid'] == 4010014625)
                     | (moncat['a_variableid'] == 3010014625))]
cns_sofa = cns_sofa.groupby(['a_patientid']).max()

In [ ]:
def func_glas(x):
    gcs = x['value']
    if (gcs>=13 and gcs<=14):
        return 1
    elif (gcs>=10 and gcs<=12):
        return 2
    elif (gcs>=6 and gcs<=9):
        return 3
    elif gcs<6:
        return 4
    elif np.isnan(gcs):
        return np.nan
    else:
        return 0
cns_sofa.loc[:, 'cns_sofa'] = cns_sofa.apply(func_glas, axis=1)

### Now the vasoactive drugs

In [ ]:
d_pharma = pd.read_parquet(f's3://icusics-db/d_pharma/d_pharma.parquet')
pharma_records3 = pd.read_parquet(
    f's3://icusics-db/pharma_records/pharma_records_h3.parquet')
pharma_records4 = pd.read_parquet(
    f's3://icusics-db/pharma_records/pharma_records_h4.parquet')
pharma_records = pd.concat([pharma_records3, pharma_records4])

drug_ids = [
    3000000060, 3000000103, 3000000183, 3000000225, 3000000225, 3000000346,
    4001005959, 4001008022, 4001006036, 4001008009, 4001008013, 4001008012,
    4001006233, 4001005925
]

filtered_df_pharma_records_cardio = pharma_records[
    pharma_records['a_pharmaid'].isin(drug_ids)]
filtered_df_pharma_records_cardio = filtered_df_pharma_records_cardio[
    filtered_df_pharma_records_cardio["time"] <= 1440]

df = df.groupby(['a_patientid', 'a_pharmaid'], as_index=False).max('givendose')

replace_values = {
    3000000183: 'NORADRENALINA',
    4001006036: 'NORADRENALINA',
    4001008022: 'NORADRENALINA',
    4001005959: 'ADRENALINA',
    4001008009: 'ADRENALINA',
    3000000103: 'ADRENALINA',
    3000000225: 'ADRENALINA',
    3000000060: 'dopamine',
    4001005925: 'dopamine',
    4001008013: 'dopamine',
    3000000346: 'dobutamina',
    4001006233: 'dobutamina',
    4001008012: 'dobutamina'
}

df['a_pharmaid'] = df['a_pharmaid'].replace(replace_values)
table = pd.pivot_table(dfcombine,
                       values='givendose',
                       index=['a_patientid'],
                       columns=['a_pharmaid'])


def cardio(x):
    if x['dopamine'] > 15 or x['ADRENALINA'] > 0.1 or x['NORADRENALINA'] > 0.1:
        return 4
    elif x['dopamine'] > 5 or (x['ADRENALINA'] <= 0.1 and x['ADRENALINA'] > 0
                               ) or (x['NORADRENALINA'] <= 0.1
                                     and x['NORADRENALINA'] > 0):
        return 3
    elif x['dopamine'] > 0 or x['dobutamina'] > 0:
        return 2
    elif x['value'] < 70:
        return 1
    elif np.isnan([
            x['value'], x['dopamine'], x['dobutamina'], x['ADRENALINA'],
            x['NORADRENALINA']
    ]).all():
        return np.nan
    else:
        return 0


meanbp = pd.read_csv("meanbp.csv")
cardio_sofa = table.merge(meanbp, on='a_patientid', how='left')
cardio_sofa['cardio_sofa'] = cardio_sofa.apply(cardio, axis=1)

## Now, merge everything

In [ ]:
sofa_score = pd.merge(cardio_sofa[['a_patientid', 'cardio_sofa']], resp+sofa[['a_patientid', 'resp_sofa']], on='a_patientid', how='outer')
sofa_score = pd.merge(sofa_score, sofa_coag[['a_patientid', 'sofa_coag']], on='a_patientid', how='outer')
sofa_score = pd.merge(sofa_score, sofa_liver[['a_patientid', 'sofa_liver']], on='a_patientid', how='outer')
sofa_score = pd.merge(sofa_score, sofa_nerv[['a_patientid', 'sofa_nervioso']], on='a_patientid', how='outer')
sofa_score = pd.merge(sofa_score, sofa_renal[['a_patientid', 'sofa_renal']], on='a_patientid', how='outer')

In [ ]:
sofa_score.loc[:, 'sofa_score'] = sofa_score.set_index('a_patientid').sum(axis=1).values

In [ ]:
icusics_db_patients = pd.read_parquet(
    "s3://icusics-db/patients/patients.parquet")
patients = icusics_db_patients[icusics_db_patients['hospital_coded'].isin(
    [3, 4])].copy()

patients.loc[:, 'icu_outcome'] = 'ALIVE'
patients.loc[patients['hospdistime'] - patients['distime'] < 1440,
             'icu_outcome'] = patients.loc[patients['hospdistime'] -
                                           patients['distime'] < 1440,
                                           'hospital_outcome']

patients.loc[:, '24h_outcome'] = 'ALIVE'
patients.loc[(patients['icu_outcome'] == 'EXITUS') &
             (patients['distime'] <= 1440), '24h_outcome'] = 'EXITUS'

patients.loc[:, '5d_outcome'] = 'ALIVE'
patients.loc[(patients['icu_outcome'] == 'EXITUS') &
             (patients['distime'] <= 1440 * 5), '5d_outcome'] = 'EXITUS'

merged = pd.merge(patients, sofa_score, on='a_patientid', how='left')

diags_h3 = pd.read_parquet('s3://icusics-db/diagnoses/diagnoses_h3.parquet')
diags_h4 = pd.read_parquet('s3://icusics-db/diagnoses/diagnoses_h4.parquet')
diags = pd.concat([diags_h3, diags_h4])

diag_to_exclude = diags[diags['referencecode'].str.startswith('S', na=False)
                        | diags['referencecode'].str.startswith('T', na=False)
                        | diags['referencecode'].str.startswith('O', na=False)]
patients_to_exclude = set(diag_to_exclude['a_patientid'])

merged = merged[~merged['a_patientid'].isin(patients_to_exclude)].copy()

pic_patients = set(
    pd.read_csv('pic_to_exclude.csv', index_col=0)['a_patientid'])
merged = merged[~merged['a_patientid'].isin(pic_patients)].copy()

merged.loc[:, 'with_antibiotics'] = False
merged.loc[
    merged['a_patientid'].isin(set(antibiotics_patients['a_patientid'])),
    'with_antibiotics'] = True


def get_age_group(x):
    if x <= 35:
        return '20-35'
    if x <= 50:
        return '36-50'
    if x <= 65:
        return '51-65'
    return '>65'


def get_obesity(x):
    if x <= 18.5:
        return 'underweight'
    if x < 25:
        return 'normal'
    if x < 30:
        return 'overweight'
    if x < 35:
        return 'obesity g1'
    if x < 40:
        return 'obesity g2'
    return 'obesity g3'


merged.loc[:, 'age_group'] = merged['age'].apply(get_age_group)
merged.loc[:, 'obesity_group'] = merged['bmi'].apply(get_obesity)

merged.to_csv('cohort.csv', index=False)